In [3]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from transformers import BartModel, BartTokenizer
import torchvision.models as models
import torchaudio
from PIL import Image

#############################################
# 1. Define the Audio Model (DeepSpeech2)   #
#############################################
class DeepSpeech2AudioModel(nn.Module):
    def __init__(self, sample_rate=16000, n_mels=128, conv_out_channels=32, 
                 rnn_hidden_size=256, num_rnn_layers=3, bidirectional=True, output_dim=128):
        """
        A DeepSpeech2-inspired model:
         - Computes a mel-spectrogram,
         - Processes it through two convolutional layers,
         - Feeds the output to a multi-layer bidirectional GRU,
         - Pools over time and projects to a fixed-dimension embedding.
        """
        super(DeepSpeech2AudioModel, self).__init__()
        self.melspec = torchaudio.transforms.MelSpectrogram(sample_rate=sample_rate, n_mels=n_mels)
        self.conv = nn.Sequential(
            nn.Conv2d(1, conv_out_channels, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(conv_out_channels),
            nn.ReLU(),
            nn.Conv2d(conv_out_channels, conv_out_channels, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(conv_out_channels),
            nn.ReLU()
        )
        # After two conv layers, the frequency dimension is roughly n_mels // 4.
        self.rnn_input_size = conv_out_channels * (n_mels // 4)
        self.rnn = nn.GRU(input_size=self.rnn_input_size,
                          hidden_size=rnn_hidden_size,
                          num_layers=num_rnn_layers,
                          batch_first=True,
                          bidirectional=bidirectional)
        rnn_output_dim = rnn_hidden_size * (2 if bidirectional else 1)
        self.fc = nn.Linear(rnn_output_dim, output_dim)
        
    def forward(self, waveform):
        # waveform: [batch, T]
        mel = self.melspec(waveform)         # [batch, n_mels, time]
        mel = mel.unsqueeze(1)               # [batch, 1, n_mels, time]
        conv_out = self.conv(mel)            # [batch, conv_out_channels, new_n_mels, new_time]
        batch_size, channels, freq, time = conv_out.size()
        conv_out = conv_out.permute(0, 3, 1, 2)  # [batch, new_time, channels, freq]
        conv_out = conv_out.contiguous().view(batch_size, time, -1)  # [batch, new_time, channels*freq]
        rnn_out, _ = self.rnn(conv_out)        # [batch, new_time, rnn_output_dim]
        pooled = rnn_out.mean(dim=1)           # [batch, rnn_output_dim]
        out = self.fc(pooled)                  # [batch, output_dim]
        return out

#############################################
# 2. Define the Multimodal Classifier       #
#############################################
class MultiModalClassifier(nn.Module):
    def __init__(self, text_model, image_model, audio_model,
                 text_feat_dim, image_feat_dim, audio_feat_dim,
                 hidden_dim, num_classes):
        """
        Combines text, image, and audio encoders. Each branch is projected
        into a common hidden space and the features are averaged (if more than one
        modality is provided) before classification.
        """
        super(MultiModalClassifier, self).__init__()
        self.text_model = text_model
        self.image_model = image_model
        self.audio_model = audio_model
        
        self.text_fc = nn.Linear(text_feat_dim, hidden_dim)
        self.image_fc = nn.Linear(image_feat_dim, hidden_dim)
        self.audio_fc = nn.Linear(audio_feat_dim, hidden_dim)
        
        self.classifier = nn.Linear(hidden_dim, num_classes)
    
    def forward(self, text_input=None, image_input=None, audio_input=None):
        features = None
        modality_count = 0
        
        if text_input is not None:
            # Remove extra key "labels" if present
            text_input_filtered = {k: v for k, v in text_input.items() if k != "labels"}
            text_outputs = self.text_model(**text_input_filtered)
            pooled_text = text_outputs.last_hidden_state.mean(dim=1)
            text_features = self.text_fc(pooled_text)
            features = text_features if features is None else features + text_features
            modality_count += 1
            
        if image_input is not None:
            image_features = self.image_model(image_input)
            image_features = self.image_fc(image_features)
            features = image_features if features is None else features + image_features
            modality_count += 1
            
        if audio_input is not None:
            audio_features = self.audio_model(audio_input)
            audio_features = self.audio_fc(audio_features)
            features = audio_features if features is None else features + audio_features
            modality_count += 1
            
        if modality_count > 1:
            features = features / modality_count
            
        logits = self.classifier(features)
        return logits

#############################################
# 3. Define Inference Functions             #
#############################################
def inference_text(model, tokenizer, text, device, max_length=128):
    """Run inference on text input only."""
    model.eval()
    encoding = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    for key in encoding:
        encoding[key] = encoding[key].to(device)
    with torch.no_grad():
        logits = model(text_input=encoding, image_input=None, audio_input=None)
    pred_id = torch.argmax(logits, dim=1).item()
    return id2label[pred_id]

def inference_image(model, image_path, transform, device):
    """Run inference on image input only."""
    model.eval()
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = model(text_input=None, image_input=image, audio_input=None)
    pred_id = torch.argmax(logits, dim=1).item()
    return id2label[pred_id]

def inference_audio(model, audio_path, device, sample_rate=16000):
    """Run inference on audio input only."""
    model.eval()
    waveform, sr = torchaudio.load(audio_path)
    if sr != sample_rate:
        waveform = torchaudio.transforms.Resample(sr, sample_rate)(waveform)
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)
    waveform = waveform.squeeze(0).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = model(text_input=None, image_input=None, audio_input=waveform)
    pred_id = torch.argmax(logits, dim=1).item()
    return id2label[pred_id]

def inference_all(model, tokenizer, text, image_path, audio_path, transform, device, sample_rate=16000, max_length=128):
    """Run inference using all three modalities."""
    model.eval()
    # Process text
    encoding = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    for key in encoding:
        encoding[key] = encoding[key].to(device)
    # Process image
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)
    # Process audio
    waveform, sr = torchaudio.load(audio_path)
    if sr != sample_rate:
        waveform = torchaudio.transforms.Resample(sr, sample_rate)(waveform)
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)
    waveform = waveform.squeeze(0).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = model(text_input=encoding, image_input=image, audio_input=waveform)
    pred_id = torch.argmax(logits, dim=1).item()
    return id2label[pred_id]

#############################################
# 4. Set Up the Model and Load Weights      #
#############################################
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define model name and load the tokenizer
model_name = "facebook/bart-base"
tokenizer = BartTokenizer.from_pretrained(model_name)

# --- Text Encoder (BART) ---
text_encoder = BartModel.from_pretrained(model_name)
text_encoder.to(device)
text_feat_dim = text_encoder.config.d_model  # Typically ~768

# --- Image Encoder (ResNet18) ---
image_encoder = models.resnet18(pretrained=True)
num_img_features = image_encoder.fc.in_features
image_encoder.fc = nn.Identity()  # Remove the classification head
image_encoder.to(device)
image_feat_dim = num_img_features  # Typically ~512

# --- Audio Encoder (DeepSpeech2-Inspired) ---
audio_encoder = DeepSpeech2AudioModel(
    sample_rate=16000, 
    n_mels=128, 
    conv_out_channels=32, 
    rnn_hidden_size=256, 
    num_rnn_layers=3, 
    bidirectional=True, 
    output_dim=128
)
audio_encoder.to(device)
audio_feat_dim = 128

# Define label list (update as needed) and create mapping
label_list = [
    "Bacterial Leaf Blight", "Brown Spot", "Healthy", "Leaf Blast", 
    "Leaf Blight", "Leaf Scald", "Leaf Smut", "Narrow Brown Spot"
]
id2label = {idx: label for idx, label in enumerate(label_list)}
num_classes = len(label_list)

# --- Instantiate the Multimodal Classifier ---
model = MultiModalClassifier(
    text_model=text_encoder,
    image_model=image_encoder,
    audio_model=audio_encoder,
    text_feat_dim=text_feat_dim,
    image_feat_dim=image_feat_dim,
    audio_feat_dim=audio_feat_dim,
    hidden_dim=512,
    num_classes=num_classes
)

# Load the saved model weights (update the path if needed)
MODEL_SAVE_PATH = "multimodal_model.pth"
if not os.path.exists(MODEL_SAVE_PATH):
    raise FileNotFoundError(f"Saved model not found at {MODEL_SAVE_PATH}")
model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=device))
model.to(device)
model.eval()

#############################################
# 5. Define Transforms for Image Inference   #
#############################################
image_inference_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

#############################################
# 6. Specify Sample Inputs and Run Inference  #
#############################################
# Update the file paths below as needed
sample_text = ("Small brown spots on leaves, spots may have yellow halos, lesions on leaf sheaths, spots may coalesce to form larger necrotic areas, leaf tip dieback, reduced grain quality")
sample_image_path = r"F:\ABDUL\ABDUL 2024\RICE PLANT DISEASE DETECTION YOLO\FINAL SOURCE CODE\MULITEMODEL_AI\IMAGES\train\Brown Spot\brown_spot (1).jpg"
sample_audio_path = r"F:\ABDUL\ABDUL 2024\RICE PLANT DISEASE DETECTION YOLO\FINAL SOURCE CODE\MULITEMODEL_AI\AUDIO\train\Brown Spot\Brown Spot_1.wav"

# Inference on each modality
predicted_label_text = inference_text(model, tokenizer, sample_text, device)
print(f"Predicted label from text: {predicted_label_text}")

predicted_label_image = inference_image(model, sample_image_path, image_inference_transform, device)
print(f"Predicted label from image: {predicted_label_image}")

predicted_label_audio = inference_audio(model, sample_audio_path, device)
print(f"Predicted label from audio: {predicted_label_audio}")

# Inference using all modalities together
predicted_label_all = inference_all(model, tokenizer, sample_text, sample_image_path, sample_audio_path,
                                    image_inference_transform, device)
print(f"Predicted label from all modalities: {predicted_label_all}")

Predicted label from text: Brown Spot
Predicted label from image: Brown Spot
Predicted label from audio: Brown Spot
Predicted label from all modalities: Brown Spot


In [2]:
import os
import time
import asyncio
from asyncio import WindowsSelectorEventLoopPolicy

import torch
import torch.nn as nn
import torchvision.transforms as transforms
from transformers import BartModel, BartTokenizer
import torchvision.models as models
import torchaudio
from PIL import Image
import g4f

#############################################
# 1. Define the Audio Model (DeepSpeech2)   #
#############################################
class DeepSpeech2AudioModel(nn.Module):
    def __init__(self, sample_rate=16000, n_mels=128, conv_out_channels=32, 
                 rnn_hidden_size=256, num_rnn_layers=3, bidirectional=True, output_dim=128):
        """
        A DeepSpeech2-inspired model:
         - Computes a mel-spectrogram,
         - Processes it through two convolutional layers,
         - Feeds the output to a multi-layer bidirectional GRU,
         - Pools over time and projects to a fixed-dimension embedding.
        """
        super(DeepSpeech2AudioModel, self).__init__()
        self.melspec = torchaudio.transforms.MelSpectrogram(sample_rate=sample_rate, n_mels=n_mels)
        self.conv = nn.Sequential(
            nn.Conv2d(1, conv_out_channels, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(conv_out_channels),
            nn.ReLU(),
            nn.Conv2d(conv_out_channels, conv_out_channels, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(conv_out_channels),
            nn.ReLU()
        )
        # After two conv layers, the frequency dimension is roughly n_mels // 4.
        self.rnn_input_size = conv_out_channels * (n_mels // 4)
        self.rnn = nn.GRU(input_size=self.rnn_input_size,
                          hidden_size=rnn_hidden_size,
                          num_layers=num_rnn_layers,
                          batch_first=True,
                          bidirectional=bidirectional)
        rnn_output_dim = rnn_hidden_size * (2 if bidirectional else 1)
        self.fc = nn.Linear(rnn_output_dim, output_dim)
        
    def forward(self, waveform):
        # waveform: [batch, T]
        mel = self.melspec(waveform)         # [batch, n_mels, time]
        mel = mel.unsqueeze(1)               # [batch, 1, n_mels, time]
        conv_out = self.conv(mel)            # [batch, conv_out_channels, new_n_mels, new_time]
        batch_size, channels, freq, time_steps = conv_out.size()
        conv_out = conv_out.permute(0, 3, 1, 2)  # [batch, new_time, channels, freq]
        conv_out = conv_out.contiguous().view(batch_size, time_steps, -1)  # [batch, new_time, channels*freq]
        rnn_out, _ = self.rnn(conv_out)        # [batch, new_time, rnn_output_dim]
        pooled = rnn_out.mean(dim=1)           # [batch, rnn_output_dim]
        out = self.fc(pooled)                  # [batch, output_dim]
        return out

#############################################
# 2. Define the Multimodal Classifier       #
#############################################
class MultiModalClassifier(nn.Module):
    def __init__(self, text_model, image_model, audio_model,
                 text_feat_dim, image_feat_dim, audio_feat_dim,
                 hidden_dim, num_classes):
        """
        Combines text, image, and audio encoders. Each branch is projected
        into a common hidden space and the features are averaged (if more than one
        modality is provided) before classification.
        """
        super(MultiModalClassifier, self).__init__()
        self.text_model = text_model
        self.image_model = image_model
        self.audio_model = audio_model
        
        self.text_fc = nn.Linear(text_feat_dim, hidden_dim)
        self.image_fc = nn.Linear(image_feat_dim, hidden_dim)
        self.audio_fc = nn.Linear(audio_feat_dim, hidden_dim)
        
        self.classifier = nn.Linear(hidden_dim, num_classes)
    
    def forward(self, text_input=None, image_input=None, audio_input=None):
        features = None
        modality_count = 0
        
        if text_input is not None:
            # Remove extra key "labels" if present
            text_input_filtered = {k: v for k, v in text_input.items() if k != "labels"}
            text_outputs = self.text_model(**text_input_filtered)
            pooled_text = text_outputs.last_hidden_state.mean(dim=1)
            text_features = self.text_fc(pooled_text)
            features = text_features if features is None else features + text_features
            modality_count += 1
            
        if image_input is not None:
            image_features = self.image_model(image_input)
            image_features = self.image_fc(image_features)
            features = image_features if features is None else features + image_features
            modality_count += 1
            
        if audio_input is not None:
            audio_features = self.audio_model(audio_input)
            audio_features = self.audio_fc(audio_features)
            features = audio_features if features is None else features + audio_features
            modality_count += 1
            
        if modality_count > 1:
            features = features / modality_count
            
        logits = self.classifier(features)
        return logits

#############################################
# 3. Define Inference Functions             #
#############################################
def inference_text(model, tokenizer, text, device, max_length=128):
    """Run inference on text input only."""
    model.eval()
    encoding = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    for key in encoding:
        encoding[key] = encoding[key].to(device)
    with torch.no_grad():
        logits = model(text_input=encoding, image_input=None, audio_input=None)
    pred_id = torch.argmax(logits, dim=1).item()
    return id2label[pred_id]

def inference_image(model, image_path, transform, device):
    """Run inference on image input only."""
    model.eval()
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = model(text_input=None, image_input=image, audio_input=None)
    pred_id = torch.argmax(logits, dim=1).item()
    return id2label[pred_id]

def inference_audio(model, audio_path, device, sample_rate=16000):
    """Run inference on audio input only."""
    model.eval()
    waveform, sr = torchaudio.load(audio_path)
    if sr != sample_rate:
        waveform = torchaudio.transforms.Resample(sr, sample_rate)(waveform)
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)
    waveform = waveform.squeeze(0).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = model(text_input=None, image_input=None, audio_input=waveform)
    pred_id = torch.argmax(logits, dim=1).item()
    return id2label[pred_id]

def inference_all(model, tokenizer, text, image_path, audio_path, transform, device, sample_rate=16000, max_length=128):
    """Run inference using all three modalities."""
    model.eval()
    # Process text
    encoding = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    for key in encoding:
        encoding[key] = encoding[key].to(device)
    # Process image
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)
    # Process audio
    waveform, sr = torchaudio.load(audio_path)
    if sr != sample_rate:
        waveform = torchaudio.transforms.Resample(sr, sample_rate)(waveform)
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)
    waveform = waveform.squeeze(0).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = model(text_input=encoding, image_input=image, audio_input=waveform)
    pred_id = torch.argmax(logits, dim=1).item()
    return id2label[pred_id]

#############################################
# 4. Set Up the Model and Load Weights      #
#############################################
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define model name and load the tokenizer
model_name = "facebook/bart-base"
tokenizer = BartTokenizer.from_pretrained(model_name)

# --- Text Encoder (BART) ---
text_encoder = BartModel.from_pretrained(model_name)
text_encoder.to(device)
text_feat_dim = text_encoder.config.d_model  # Typically ~768

# --- Image Encoder (ResNet18) ---
image_encoder = models.resnet18(pretrained=True)
num_img_features = image_encoder.fc.in_features
image_encoder.fc = nn.Identity()  # Remove the classification head
image_encoder.to(device)
image_feat_dim = num_img_features  # Typically ~512

# --- Audio Encoder (DeepSpeech2-Inspired) ---
audio_encoder = DeepSpeech2AudioModel(
    sample_rate=16000, 
    n_mels=128, 
    conv_out_channels=32, 
    rnn_hidden_size=256, 
    num_rnn_layers=3, 
    bidirectional=True, 
    output_dim=128
)
audio_encoder.to(device)
audio_feat_dim = 128

# Define label list (update as needed) and create mapping
label_list = [
    "Bacterial Leaf Blight", "Brown Spot", "Healthy", "Leaf Blast", 
    "Leaf Blight", "Leaf Scald", "Leaf Smut", "Narrow Brown Spot"
]
id2label = {idx: label for idx, label in enumerate(label_list)}
num_classes = len(label_list)

# --- Instantiate the Multimodal Classifier ---
model = MultiModalClassifier(
    text_model=text_encoder,
    image_model=image_encoder,
    audio_model=audio_encoder,
    text_feat_dim=text_feat_dim,
    image_feat_dim=image_feat_dim,
    audio_feat_dim=audio_feat_dim,
    hidden_dim=512,
    num_classes=num_classes
)

# Load the saved model weights (update the path if needed)
MODEL_SAVE_PATH = "multimodal_model.pth"
if not os.path.exists(MODEL_SAVE_PATH):
    raise FileNotFoundError(f"Saved model not found at {MODEL_SAVE_PATH}")
model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=device))
model.to(device)
model.eval()

#############################################
# 5. Define Transforms for Image Inference   #
#############################################
image_inference_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

#############################################
# 6. Specify Sample Inputs and Run Inference  #
#############################################
# Update the file paths below as needed
sample_text = (
    "Small brown spots on leaves, spots may have yellow halos, lesions on leaf sheaths, "
    "spots may coalesce to form larger necrotic areas, leaf tip dieback, reduced grain quality"
)
sample_image_path = r"F:\ABDUL\ABDUL 2024\RICE PLANT DISEASE DETECTION YOLO\FINAL SOURCE CODE\MULITEMODEL_AI\IMAGES\train\Brown Spot\brown_spot (1).jpg"
sample_audio_path = r"F:\ABDUL\ABDUL 2024\RICE PLANT DISEASE DETECTION YOLO\FINAL SOURCE CODE\MULITEMODEL_AI\AUDIO\train\Brown Spot\Brown Spot_1.wav"

# Inference on individual modalities
predicted_label_text = inference_text(model, tokenizer, sample_text, device)
print(f"Predicted label from text: {predicted_label_text}")

predicted_label_image = inference_image(model, sample_image_path, image_inference_transform, device)
print(f"Predicted label from image: {predicted_label_image}")

predicted_label_audio = inference_audio(model, sample_audio_path, device)
print(f"Predicted label from audio: {predicted_label_audio}")

# Inference using all modalities together
predicted_label_all = inference_all(model, tokenizer, sample_text, sample_image_path, sample_audio_path,
                                    image_inference_transform, device)
print(f"Predicted label from all modalities: {predicted_label_all}")

#############################################
# 7. Generate Suggestion Report via GPT-4   #
#############################################
# Set the event loop policy for Windows
asyncio.set_event_loop_policy(WindowsSelectorEventLoopPolicy())

def generate_response(user_input):
    """Generate a response using GPT-4 via g4f."""
    try:
        response = g4f.ChatCompletion.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": user_input}],
            temperature=0.6,
            top_p=0.9
        )
        return response.strip() if response else "No response generated."
    except Exception as e:
        return f"Error generating response: {e}"

def generate_suggestion_report(disease_label):
    """
    Generate a detailed, step-by-step suggestion report for the detected disease.
    The report includes:
      1. What it is
      2. Why it occurs
      3. How to overcome
      4. Fertilizer Recommendations
    The report is provided in both English and Tamil.
    """
    prompt = (
        f"Provide a detailed, step-by-step suggestion report for the plant disease '{disease_label}' detected. "
        "The report should have the following sections with clear headings:\n"
        "1. What it is: Explain what the disease is in simple terms.\n"
        "2. Why it occurs: Describe the causes and contributing factors for the disease.\n"
        "3. How to overcome: Provide a step-by-step guide on how to manage and overcome the disease.\n"
        "4. Fertilizer Recommendations: Suggest the type of fertilizer and application methods suitable for this condition.\n"
        "Please provide the report in both English and Tamil, with each section written in both languages."
    )
    return generate_response(prompt)

# Generate suggestion report based on the detected disease
detected_disease = predicted_label_all
print(f"\nDetected disease: {detected_disease}")
suggestion_report = generate_suggestion_report(detected_disease)
print("\nSuggestion Report:")
print(suggestion_report)

# Save the suggestion report to a text file (download the report)
report_filename = "suggestion_report.txt"
with open(report_filename, "w", encoding="utf-8") as f:
    f.write(suggestion_report)
print(f"Suggestion report saved to {report_filename}")


Predicted label from text: Brown Spot
Predicted label from image: Brown Spot
Predicted label from audio: Brown Spot
Predicted label from all modalities: Brown Spot

Detected disease: Brown Spot

Suggestion Report:
Sure, here's a detailed report on managing the plant disease 'Brown Spot' in both English and Tamil:

---

### What it is: என்னுடையது
**English:**
Brown Spot is a fungal disease that affects various plants, causing brown, circular spots on leaves, stems, and sometimes fruits. These spots can merge and lead to significant damage, affecting the plant's overall health and yield.

**Tamil:**
பிரவுன் ஸ்பாட் ஒரு பூச்சி நோய் ஆகும், இது பல்வேறு தாவரங்களை பாதிப்பது, இதனால் இலை, கிளைகள், மற்றும் சில மாட்டுகளில் பிடிப்புகள் ஏற்படுகின்றன. இந்த பிடிப்புகள் இணைந்து முக்கிய சேதம் செய்யக்கூடியது, தாவரத்தின் மொத்த ஆரோக்கியத்தையும் உற்பத்தியையும் பாதிப்பது.

### Why it occurs: எண்ணினால்
**English:**
Brown Spot occurs due to fungal infections, often caused by high humidity and poor air circulat

C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\asyncio\events.py:84: UserWarning: Curlm alread closed! quitting from process_data
  self._context.run(self._callback, *self._args)
